Streaming model from CHI@UC object-store to docker jupyter instead of saving the model locally and running it.

In [ ]:
curl https://rclone.org/install.sh | sudo bash

In [ ]:
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf

In [ ]:
mkdir -p ~/.config/rclone

In [ ]:
nano  ~/.config/rclone/rclone.conf

Paste the below into rclone file. The values YOUR_USER_ID can be found in Users inside Identity and YOUR_APP_CRED_ID, YOUR_APP_CRED_SECRET are the credentials unique to the project, created on Chameleon App Credentials under Identity. (We can't access any object store even if it's in public access without credentials. Even if we're only reading the data)


We'll add the below snippet of code into the rclone file


In [ ]:
[chi_uc]
type = swift
user_id = YOUR_USER_ID
application_credential_id = YOUR_APP_CRED_ID
application_credential_secret = YOUR_APP_CRED_SECRET
auth = https://chi.uc.chameleoncloud.org:5000/v3
region = CHI@UC

Mounting the object store container to VM

In [ ]:
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
rclone mount chi_uc:object-persist-project32 /mnt/object --read-only --allow-other --daemon

Do similar steps to mount the training data object in chi-tacc

Restart the container if it's already running

docker stop jupyter 

In [ ]:
docker run -d --rm \
      --gpus all \
      --shm-size 64G \
      -v /home/cc/MLOps/model_serving/system_optimizations:/home/jovyan/work \
      --mount type=bind,source=/mnt/object,target=/mnt/object,readonly \
      --mount type=bind,source=/mnt/object_group,target=/mnt/object_group,readonly \
      -p 8888:8888   --user $(id -u):$(id -g) \
      --name jupyter \
      jupyter-onnx

Now, the model is available in Jupyter at /mnt/object/my_model.pth in the instance

In [ ]:
docker logs jupyter

Find a line similar to the below and change the ip to our floating ip

In [ ]:
http://127.0.0.1:8888/lab?token=1150b6ba3fe76e40359a9b4a3b0df2015e847e91305ee852

The rest of the steps are in model_optimizations